In [1]:
from keras.layers import Input,Lambda,Dense,Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
import os
import cv2
import tensorflow as tf
from keras.applications.vgg16 import decode_predictions


In [2]:
print(tf.__version__)

2.4.1


In [3]:
def get_file_size(file_path):
    size=os.path.getsize(file_path)
    return size

In [4]:
categories=['train','test']
categories1=['without_mask','with_mask']


In [5]:
def covert_bytes(size,unit=None):
    if unit=="KB":
        return print("File size"+str(round(size/1024,3))+'Kilobytes')
    elif unit=="MB":
        return print("File size"+str(round(size/1024,3))+'Megabytes')
    else:
        return print("File size"+str(round(size/1024,3))+'bytes')
    

In [6]:
data1=[]
for category in categories1:
    path=os.path.join('Dataset',category)
    label1=categories1.index(category)
    for file in os.listdir(path):
       img_path=os.path.join(path,file)
       img=cv2.imread(img_path)
       img=cv2.resize(img,(224,224))
       data1.append([img,label1])
    

In [7]:
data=[]
cate=[]
for category in categories:
    path=os.path.join('Dataset',category)
    for file in os.listdir(path):
        if not file.startswith('.'):
            img_path=os.path.join(path,file)
            cate.append(file)
            label=cate.index(file)
            files=os.listdir(img_path)
            for i in files:
                p=os.path.join(img_path,i)
                img_path=os.path.join(path,file)
                #print(p)
                img=cv2.imread(p)
                img=cv2.resize(img,(224,224))
                data.append([img,label])


In [8]:
import random

In [9]:
random.shuffle(data1)

In [10]:
x1=[]
y1=[]
for features1,label1 in data1:
    x1.append(features1)
    y1.append(label1)  

In [11]:
import numpy as np

In [12]:
x1=np.array(x1)
y1=np.array(y1)

In [13]:
x1.shape
y1.shape

(1965,)

In [14]:
x1=x1/255

In [15]:
x1[0]

array([[[0.        , 0.00784314, 0.05098039],
        [0.        , 0.00784314, 0.05098039],
        [0.        , 0.00784314, 0.05098039],
        ...,
        [0.        , 0.00392157, 0.01960784],
        [0.        , 0.00392157, 0.01960784],
        [0.        , 0.00392157, 0.01960784]],

       [[0.        , 0.00784314, 0.05098039],
        [0.        , 0.00784314, 0.05098039],
        [0.        , 0.00784314, 0.05098039],
        ...,
        [0.        , 0.00392157, 0.01960784],
        [0.        , 0.00392157, 0.01960784],
        [0.        , 0.00392157, 0.01960784]],

       [[0.        , 0.00784314, 0.05098039],
        [0.        , 0.00784314, 0.05098039],
        [0.        , 0.00784314, 0.05098039],
        ...,
        [0.        , 0.00392157, 0.01960784],
        [0.        , 0.00392157, 0.01960784],
        [0.        , 0.00392157, 0.01960784]],

       ...,

       [[0.07058824, 0.07058824, 0.07058824],
        [0.07058824, 0.07058824, 0.07058824],
        [0.07058824, 0

In [16]:
from sklearn.model_selection  import train_test_split
x_train1,x_test1,y_train1,y_test1=train_test_split(x1,y1,test_size=0.2)

In [17]:
x_train1.shape
x_test1.shape

(393, 224, 224, 3)

In [18]:
from tensorflow.keras.applications.vgg16 import VGG16

In [19]:
vgg1=VGG16()

In [20]:
vgg1.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [21]:
from keras import Sequential

In [22]:
model1=Sequential()

In [23]:
for layer in vgg1.layers[:-1]:
    model1.add(layer)

In [24]:
model1.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)       2

In [25]:
for layer in model1.layers:
    layer.trainable=False

In [26]:
model1.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)       2

In [27]:
from keras.layers import Dense

In [28]:
model1.add(Dense(1,activation='sigmoid'))

In [29]:
model1.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)       2

In [30]:
model1.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['accuracy'])

In [31]:
model1.fit(x_train1,y_train1,epochs=3,validation_data=(x_test1,y_test1))

Epoch 1/3
50/50 [==============================] - 448s 9s/step - loss: 0.6506 - accuracy: 0.6097 - val_loss: 0.2995 - val_accuracy: 0.9567
Epoch 2/3
50/50 [==============================] - 467s 9s/step - loss: 0.2629 - accuracy: 0.9589 - val_loss: 0.1896 - val_accuracy: 0.9720
Epoch 3/3
50/50 [==============================] - 465s 9s/step - loss: 0.1733 - accuracy: 0.9768 - val_loss: 0.1592 - val_accuracy: 0.9669


In [32]:
h=(int(len(cate)/2))
del cate[:h]
print(cate)
print(len(cate))

['chaya', 'manja', 'queany', 'rakesh', 'sam', 'sankalpa', 'sapana', 'shradda', 'shreya', 'sowmya', 'suprabha', 'swathi', 'thrupthi', 'vaishali', 'vaishu']
15


In [33]:
random.shuffle(data)

In [34]:
x=[]
y=[]
for features,label in data:
    x.append(features)
    y.append(label)    

In [35]:
x=np.array(x)
y=np.array(y)

In [36]:
x.shape
y.shape

(522,)

In [37]:
x=x/255

In [38]:
x[0]

array([[[0.16862745, 0.27058824, 0.23137255],
        [0.12156863, 0.17254902, 0.16862745],
        [0.15686275, 0.21960784, 0.22745098],
        ...,
        [0.10196078, 0.13333333, 0.13333333],
        [0.21176471, 0.24313725, 0.23921569],
        [0.11764706, 0.16470588, 0.15294118]],

       [[0.16078431, 0.18431373, 0.20392157],
        [0.09411765, 0.15686275, 0.16470588],
        [0.1254902 , 0.19607843, 0.16862745],
        ...,
        [0.12156863, 0.15686275, 0.14509804],
        [0.10980392, 0.17647059, 0.16078431],
        [0.08627451, 0.1254902 , 0.09411765]],

       [[0.15294118, 0.17647059, 0.19607843],
        [0.11372549, 0.10980392, 0.1254902 ],
        [0.08235294, 0.09019608, 0.07058824],
        ...,
        [0.12156863, 0.16078431, 0.14509804],
        [0.12941176, 0.19607843, 0.18039216],
        [0.12941176, 0.2       , 0.16470588]],

       ...,

       [[0.09803922, 0.10588235, 0.10588235],
        [0.12941176, 0.14117647, 0.13333333],
        [0.07058824, 0

In [39]:
from sklearn.model_selection  import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2)

In [40]:
x_train.shape
x_test.shape

(105, 224, 224, 3)

In [41]:
vgg=VGG16()

In [42]:
vgg.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [43]:
from keras import Sequential

In [44]:
model=Sequential()

In [45]:
for layer in vgg.layers[:-1]:
    model.add(layer)

In [46]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)      

In [47]:
for layer in model.layers:
    layer.trainable=False

In [48]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)      

In [49]:
model.add(Dense(len(cate),activation='softmax'))

In [50]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)      

In [51]:
model.compile(optimizer='Adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [52]:
model.fit(x_train,y_train,epochs=5,validation_data=(x_test,y_test))

Epoch 1/5
14/14 [==============================] - 112s 8s/step - loss: 2.2497 - accuracy: 0.4458 - val_loss: 1.2744 - val_accuracy: 0.6476
Epoch 2/5
14/14 [==============================] - 118s 9s/step - loss: 1.5070 - accuracy: 0.6598 - val_loss: 0.9410 - val_accuracy: 0.7333
Epoch 3/5
14/14 [==============================] - 127s 9s/step - loss: 0.9372 - accuracy: 0.7352 - val_loss: 0.8310 - val_accuracy: 0.7048
Epoch 4/5
14/14 [==============================] - 124s 9s/step - loss: 0.8562 - accuracy: 0.7107 - val_loss: 0.7751 - val_accuracy: 0.7238
Epoch 5/5
14/14 [==============================] - 123s 9s/step - loss: 0.8813 - accuracy: 0.6608 - val_loss: 0.7101 - val_accuracy: 0.7714


In [53]:
def detect_face_mask(img):
    y_pred=model.predict_classes(img.reshape(1,224,224,3))
    return y_pred

In [54]:
def detect_face_mask1(img):
    y_pred=model1.predict_classes(img.reshape(1,224,224,3))
    return y_pred[0][0]

In [55]:
def draw_label(img,text,pos,bg_color):
    text_size=cv2.getTextSize(text,cv2.FONT_HERSHEY_SIMPLEX,1,cv2.FILLED)
    end_x=pos[0]+text_size[0][0]+2
    end_y=pos[1]+text_size[0][1]-2
    cv2.rectangle(img,pos,(end_x,end_y),bg_color,cv2.FILLED)
    cv2.putText(img,text,pos,cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),1,cv2.LINE_AA)

In [56]:
haar=cv2.CascadeClassifier('haarcascade_eye.xml')
haar1=cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

In [57]:
def detect_face(img):
    coord=haar.detectMultiScale(img)
    return coord

In [58]:
def detect_face1(img):
    coord=haar1.detectMultiScale(img)
    return coord

In [97]:
KERAS_MODEL_NAME="tf_model_facerecognition.h5"

In [98]:
model.save(KERAS_MODEL_NAME)

In [99]:
covert_bytes(get_file_size(KERAS_MODEL_NAME),"MB")

File size525256.117Megabytes


In [100]:
test_loss,test_acc=model.evaluate(x_test,y_test,verbose=2)

4/4 - 34s - loss: 0.7101 - accuracy: 0.7714


In [101]:
TF_LITE_MODEL_FILE_NAME="tf_lite_model.tflite"

In [ ]:
tf_lite_converter=tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model=tf_lite_converter.convert()

INFO:tensorflow:Assets written to: C:\Users\User\AppData\Local\Temp\tmpz_16t_xk\assets


INFO:tensorflow:Assets written to: C:\Users\User\AppData\Local\Temp\tmpz_16t_xk\assets


In [ ]:
tflite_model_name=TF_LITE_MODEL_FILE_NAME
open(tflite_model_name,"wb").write(tflite_model)

In [ ]:
covert_bytes(get_file_size(TF_LITE_MODEL_FILE_NAME),"KB")

In [ ]:
interpreter=tf.lite.Interpreter(model_path=TF_LITE_MODEL_FILE_NAME)
input_details=interpreter.get_input_details()
output_details=interpreter.get_output_details()
print("Input shape:",input_details[0]['shape'])
print("Input Type:",input_details[0]['dtype'])
print("Output shape:",output_details[0]['shape'])
print("output dtype:",output_details[0]['dtype'])

In [ ]:
y_test.shape[0]


In [ ]:
interpreter.resize_tensor_input(input_details[0]['index'],(x_test.shape[0],224,224))
interpreter.resize_tensor_input(output_details[0]['index'],(y_test.shape[0],len(cate)))
interpreter.allocate_tensors()
input_details=interpreter.get_input_details()
output_details=interpreter.get_output_details()
print("Input shape:",input_details[0]['shape'])
print("Input Type:",input_details[0]['dtype'])
print("Output shape:",output_details[0]['shape'])
print("output dtype:",output_details[0]['dtype'])

In [ ]:
x_test.dtype

In [72]:
test_imgs_numpy=np.array(x_test,dtype=np.float32)

In [90]:
interpreter.set_tensor(input_details[0]['index'],test_imgs_numpy)
interpreter.invoke()
tflite_model_predictions=interpreter.get_tensor(output_details[0]['index'])
prediction_classes=np.argmax(tflite_model_predictions,axis=1)

ValueError: Cannot set tensor: Dimension mismatch. Got 4 but expected 3 for input 0.

In [56]:
cap=cv2.VideoCapture(0)

In [ ]:
while True:
    ret,frame=cap.read()
    img=cv2.resize(frame,(224,224))
    coord=detect_face(cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY))
    coord1=detect_face1(cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY))
    for x,y,w,h in coord:
        cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2)
    for x,y,w,h in coord1:
        cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2)
    y_pred1=detect_face_mask1(img)
    y_pred=int(detect_face_mask(img))
    str1=""
    if y_pred1==0:
        str1="No Mask "+cate[y_pred]
    else:
        str1="With Mask "+cate[y_pred]
    print(y_pred)
    draw_label(frame,str1,(30,30),(0,0,255))
    cv2.imshow('window',frame)
    if cv2.waitKey(1) & 0xFF== ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

C:\Users\User\anaconda3\lib\site-packages\tensorflow\python\keras\engine\sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


10
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
12
14
14
14
14
12
12
14
12
12
12
12
12
12
12
12
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
12
12
12
12
12
14
12
12
12
12
12
12
12
12
12
10
10
12
12
12
12
14
14
12
12
14
12
14
12
12
12
12
12
12
14
14
10
10
10
10
14
14
10
14
14
14
12
12
12
12
12
12
14
12
12
12
13
12
12
12
12
12
12
14
14
12
12
12
12
14
14
12
12
13
10
13
14
12
13
13
13
12
12
13
12
12
12
12
12
12
12
12
12
12
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
12
14
12
14
14
14
12
14
14
13
12
14
14
14
14
14
14
14
14
10
10
14
14
14
12
12
12
12
12
12
12
12
13
12
13
13
13
13
13
13
13
12
13
12
12
12
12
12
12
12
12
12
12
12
14
12
13
13
13
13
13
13
13
12
12
13
12
12
12
12
12
12
12
13
12
12
12
12
12
12
14
14
14
14
14
14
14
14
14
12
14
14
14
13
13
13
13
13
1

10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
14
14
14
14
14
14
14
14
10
14
10
10
10
10
10
10
10
10
10
14
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
14
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
14
10
10
14
14
14
14
14
14
14
10
14
14
10
10
10
10
10
10
10
10
10
10
14
14
14
10
10
10
14
10
10
14
10
14
14
14
14
14
14
14
10
10
10
10
10
10
10
10
10
10
10
10
14
14
14
14
14
14
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
14
10
10
1

14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
10
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
10
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
